In [67]:
import methylize
import methylcheck
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

In [68]:
betas, meta = methylcheck.load_both('../data/asthma') #load in the beta values and metadata
betas = betas.T

INFO:methylcheck.load_processed:Found several meta_data files; attempting to match each with its respective beta_values files in same folders.
INFO:methylcheck.load_processed:Multiple meta_data found. Only loading the first file.
INFO:methylcheck.load_processed:Loading 40 samples.
Files: 100%|██████████| 1/1 [00:00<00:00, 25.02it/s]
INFO:methylcheck.load_processed:loaded data (485512, 40) from 1 pickled files (0.046s)
INFO:methylcheck.load_processed:Transposed data and reordered meta_data so sample ordering matches.
INFO:methylcheck.load_processed:meta.Sample_IDs match data.index (OK)


In [69]:
betas = betas.dropna(how='any', axis='columns')
betas = betas.sample(100, axis = 1)

In [70]:
betas.isna().sum().sum()

0

In [71]:
merge = betas.merge(meta[['Sample_ID', 'disease', 'age']], left_index=True, right_on='Sample_ID').drop(columns='Sample_ID')
merge.head()

,cg22865720,cg02222290,cg06659958,cg17056607,cg17508204,cg10376306,cg02933362,cg25004108,cg18129755,cg13879247,...,cg14597637,cg15690542,cg22960501,cg08069931,cg22038796,cg01359967,cg01072121,cg03638180,disease,age
20,0.038,0.019,0.964,0.316,0.013,0.751,0.176,0.936,0.037,0.939,...,0.961,0.085,0.057,0.084,0.016,0.952,0.016,0.015,Healthy,65
38,0.036,0.020,0.968,0.384,0.015,0.922,0.076,0.943,0.039,0.942,...,0.957,0.097,0.055,0.089,0.022,0.939,0.017,0.019,Asthmatic,19
37,0.025,0.027,0.973,0.445,0.016,0.897,0.040,0.949,0.025,0.945,...,0.965,0.091,0.048,0.103,0.017,0.950,0.019,0.018,Asthmatic,14
32,0.018,0.024,0.968,0.435,0.015,0.859,0.038,0.943,0.025,0.950,...,0.962,0.088,0.049,0.092,0.024,0.954,0.018,0.014,Asthmatic,15
23,0.114,0.023,0.968,0.302,0.013,0.757,0.036,0.954,0.134,0.934,...,0.955,0.080,0.044,0.069,0.027,0.960,0.019,0.018,Asthmatic,8


In [72]:
lin = LinearRegression().fit(merge.drop(columns=['disease', 'age']).to_numpy(),merge.age)

In [73]:
X = np.asarray(merge.drop(columns=['disease', 'age']))
Y = np.asarray(merge.age).astype(float)

In [74]:
from sklearn.feature_selection import f_regression

f = f_regression(X, Y)
pvals = f[1]

In [75]:
test_results2 = methylize.diff_meth_pos(betas, merge.age, regression_method="linear", export=False)
test_results2

  0%|          | 0/100 [00:00<?, ?it/s]

,Coefficient,StandardError,PValue,95%CI_lower,95%CI_upper,Rsquared,FDR_QValue
cg12608132,0.004990,0.000822,4.568175e-07,0.700791,0.702428,0.492257,0.000046
cg06695800,-0.002052,0.000407,1.180195e-05,-0.633269,-0.632312,0.400424,0.000590
cg20136609,-0.001693,0.000405,1.673450e-04,-0.561328,-0.560239,0.314478,0.005578
cg22672060,0.004328,0.001067,2.381218e-04,0.548327,0.551244,0.302266,0.005953
cg12800915,-0.001935,0.000522,6.641518e-04,-0.516222,-0.514720,0.265711,0.009916
...,...,...,...,...,...,...,...
cg10207240,-0.000172,0.001085,8.749784e-01,-0.027813,-0.023562,0.000660,0.911436
cg03638180,0.000001,0.000021,9.432915e-01,0.011574,0.011656,0.000135,0.972465
cg14814018,-0.000003,0.000075,9.679276e-01,-0.006712,-0.006419,0.000043,0.987681
cg07031748,-0.000002,0.000069,9.785637e-01,-0.004522,-0.004253,0.000019,0.988448


In [76]:
pvals = ['%.10f' % i for i in pvals]
methp = ['%.10f' % i for i in test_results2.PValue]

In [77]:
same = 0
for i, j in zip(sorted(pvals), sorted(methp)):
    if i == j:
        same += 1
same

100